### Import Required Modules

In [1]:
import os
import glob
import itertools
from pathlib import Path
import boto3
import pandas as pd
from deltalake import DeltaTable
from deltalake.writer import write_deltalake

### Create Delta Table

In [ ]:
for i in range(0, 10_000):
    df = pd.DataFrame({'x': [j for j in range(i * 100, (i + 1) * 100)]})
    if i == 0:
        write_deltalake('s3://delta-rs-playground/test', df)
    else:
        write_deltalake('s3://delta-rs-playground/test', df, mode='append')

### Utility Functions

In [2]:
CLIENT = boto3.client('s3')

def upload(local_path, bucket, gcs_path):
    assert os.path.isdir(local_path)
    for local_file in itertools.chain(glob.glob(local_path + '/**'), glob.glob(local_path + '/.**')):
        if not os.path.isfile(local_file):
            upload(local_file, bucket, gcs_path + "/" + os.path.basename(local_file))
        else:
            remote_path = os.path.join(gcs_path, os.path.basename(local_file))
            CLIENT.upload_file(local_file, bucket, remote_path)

### Upload Table to S3

In [4]:
upload('../data/local_table/', 'delta-rs-playground', 'test')